## 1-0: Connect to MySQL and create database

In [91]:
# mycursor.execute("DROP DATABASE mydatabase;")

In [92]:
import csv
import json
import mysql.connector

# please modify hw4_config.json to your own config file
with open('hw5_config.json') as f:
    config = json.load(f)

host = config['host']
user = config['user']
passwd = config['passwd']
dbms_student_path = config['dbms_student_path']
new_student_path = config['new_student_path']

mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=passwd
)

print(mydb)

mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE mydatabase")
mycursor.execute("USE mydatabase")

## 1-1: Input data into student table

In [93]:
# mycursor.execute("DROP TABLE student;")

In [94]:
mydb.consume_results()
with open(dbms_student_path, 'r', newline='', encoding='big5') as csv_data:
    reader = csv.reader(csv_data)
    columns = next(reader) # get first row 
    print(columns)

    # create table
    table_name = "student"
    sql_create_table = f"CREATE TABLE {table_name} ("
    column_definitions = ", ".join(f"`{col}` VARCHAR(100)" for col in columns)
    sql_create_table += column_definitions + ");"
    mycursor.execute(sql_create_table)

    placeholders = ", ".join(["%s"] * len(columns))
    sql_insert = f"INSERT INTO {table_name} ({', '.join(f'`{col}`' for col in columns)}) VALUES ({placeholders})"

    # Insert data into table row by row
    for row in reader: 
        mycursor.execute(sql_insert, row)
        mydb.commit()


['身份', '系所', '年級', '學號', '姓名', '信箱', '班別']


## 1-2: Read the information about yourself

In [95]:
mydb.consume_results()
mycursor.execute("SELECT * FROM student WHERE student.學號='R12921059';")
result = mycursor.fetchall()
print (result)

[('學生', '電機所      ', '1', 'R12921059', '鄧雅文 (TENG, YA-WEN)', 'r12921059@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]


## 1-3: Update your own value of the "身份" column 

In [96]:
mydb.consume_results()
mycursor.execute("UPDATE student SET 身份 = '特優生' WHERE 學號 = 'R12921059' AND 身份 = '學生';")
mycursor.execute("SELECT * FROM student WHERE student.學號='R12921059';")
result = mycursor.fetchall()
print (result)

[('特優生', '電機所      ', '1', 'R12921059', '鄧雅文 (TENG, YA-WEN)', 'r12921059@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]


## 1-4: Insert three new students 

In [97]:
mydb.consume_results()
with open(new_student_path, 'r', newline='', encoding='utf-8-sig') as csv_data:
    reader = csv.reader(csv_data)
    columns = next(reader) # get first row
    
    table_name = "student"
    placeholders = ", ".join(["%s"] * len(columns))
    sql_insert = f"INSERT INTO {table_name} ({', '.join(f'`{col}`' for col in columns)}) VALUES ({placeholders})"

    # Insert data into table row by row
    for row in reader: 
        mycursor.execute(sql_insert, row)
        mydb.commit() 

## 1-5: Prepared select statement 
Reference: https://zetcode.com/db/mysqlpython/

In [98]:
mydb.consume_results()
query_prepare = "SELECT * FROM student "+\
                "WHERE student.`學號`= %s;"

mycursor.execute(query_prepare, ("R10123456",))
result1 = mycursor.fetchall()
print (result1)

mycursor.execute(query_prepare, ('B09987653',))
result2 = mycursor.fetchall()
print (result2)

mycursor.execute(query_prepare, ('R11123001',))
result3 = mycursor.fetchall()
print (result3)

[('旁聽生', '電機所', '2', 'R10123456', '小紅', 'r10123456@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]
[('學生', '物理系', '3', 'B09987653', '小黃', 'b09987653@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]
[('觀察者', '電信所', '1', 'R11123001', '小綠', 'r11123001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]


## 1-6: Group information

In [99]:
mycursor.execute("ALTER TABLE student ADD `group` INT DEFAULT 0, ADD `captain` BOOLEAN DEFAULT FALSE;")

query_update_group = "UPDATE student " +\
                     "SET `group`= 7 "+\
                     "WHERE `學號`= %s;"
list_update_group = ["R12921059","R12921093","R12922116","B09502132"]
for id in list_update_group:
    mycursor.execute(query_update_group, (id,))
    mydb.commit() 


query_update_captain = "UPDATE student " +\
                       "SET captain=1 "+\
                       "WHERE `學號`= %s;"
mycursor.execute(query_update_captain, ("R12921059",))
mydb.commit() 


query_select_group = "SELECT * " +\
                     "FROM student "+\
                     "WHERE student.`group`= 7;"
mycursor.execute(query_select_group)
result = mycursor.fetchall()
print (result)

[('學生', '機械系      ', '4', 'B09502132', '周哲瑋 (CHOU,CHE-WEI)', 'b09502132@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 7, 0), ('特優生', '電機所      ', '1', 'R12921059', '鄧雅文 (TENG, YA-WEN)', 'r12921059@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 7, 1), ('學生', '電機所      ', '1', 'R12921093', '吳吉加 (CHI-CHIA WU)', 'r12921093@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 7, 0), ('學生', '資工所      ', '1', 'R12922116', '王偉力 (WANG, WEI-LI)', 'r12922116@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 7, 0)]


## 1-7: Close the database connection
Important, otherwise part2 will fail.

In [100]:
mydb.close()